In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import analyze_utils

# https://wandb.ai/jack-morris/emb-inv-1/runs/ebb31d91810c4b62d2b55b5382e8c7ea/logs?workspace=user-jxmorris12
checkpoint_folder = '/home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea'
args_str = '--per_device_train_batch_size 128 --per_device_eval_batch_size 128 --max_seq_length 32 --model_name_or_path t5-base --embedder_model_name gtr_base --num_repeat_tokens 16 --embedder_no_grad True --exp_group_name mar17-baselines --learning_rate 0.0003 --freeze_strategy none --embedder_fake_with_zeros False --use_frozen_embeddings_as_input False --num_train_epochs 24 --max_eval_samples 500 --eval_steps 25000 --warmup_steps 100000 --bf16=1 --use_wandb=1'
trainer = analyze_utils.load_trainer(checkpoint_folder, args_str)

trainer.evaluate()

Set train_args.dataloader_num_workers = 4
Loading trainer for analysis – setting --do_eval=1


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output -> <pad> The m<unk>lbies wase wyst bograge; And the sliths and toms wy</s>
================ End trainer sanity check ================


to the character of the skull, which are relatively smooth and untutuous in the case of infant sutures. Unlike the sutures,
to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



individual from the Southern Hemisphere to win the Winter Olympic relay gold medal, and was also part of the Australian Short Track team, which won gold
individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



the same rights as men, and 75% agreed that they should be protected from discrimination. Among the other 15%, gay workers are no longer exp
the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H


evaluation_loop


{'eval_loss': 1.0522613525390625,
 'eval_accuracy': 0.7386067708333334,
 'bleu_score': 31.876985502984844,
 'emb_cos_sim': 0.9062262773513794,
 'eval_perplexity': 2.8641205867926014,
 'eval_runtime': 3.8306,
 'eval_samples_per_second': 130.527,
 'eval_steps_per_second': 1.044}

In [12]:
from typing import List
import torch

def encode(s: str) -> torch.Tensor:
    t = trainer.embedder_tokenizer(s, return_tensors='pt')
    print(t)
    t = t.to(trainer.args.device)
    with torch.no_grad():
        e = trainer.call_embedding_model(
            input_ids=t['input_ids'],
            attention_mask=t['attention_mask'],
        )
    return e

def decode(t: torch.Tensor) -> List[str]:
    trainer.model.use_frozen_embeddings_as_input = True
    decoded_ids = trainer.model.generate(
        inputs={
            'embedder_input_ids': torch.tensor([1]),
            'embedder_attention_mask': torch.tensor([1]),
            'frozen_embeddings':t,
        },
        generation_kwargs={
            'early_stopping': False, 
            'num_beams': 1, 
            'do_sample': False, 
            'max_length': 32
        }
    )
    return trainer.embedder_tokenizer.batch_decode(decoded_ids)

decode(encode("If I were a rich man Ya ba dibba dibba dibba dibba dibba dibba dum"))

{'input_ids': tensor([[ 156,   27,  130,    3,    9, 2354,  388, 4701, 4698, 1227,  115,  115,
            9, 1227,  115,  115,    9, 1227,  115,  115,    9, 1227,  115,  115,
            9, 1227,  115,  115,    9, 1227,  115,  115,    9,  146,   51,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


['<pad> If I were a rich man I would be a rich man sir b b b b b b b ']

In [16]:
decode(encode("Where is LeBron James from?"))

{'input_ids': tensor([[ 2840,    19,   312, 22780,    29,  2549,    45,    58,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


['<pad> LeBron James from the state of Florida LeBron James is a basketballer who was born in the city of Topeka, Ohio and']

In [161]:
a = encode("Where is Lebon James from?")
b = encode("What is the capital of California?")

decode( (a+b)/2 )

{'input_ids': tensor([[2840,   19,  312, 5407, 2549,   45,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[ 363,   19,    8, 1784,   13, 1826,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


['<pad> The Head of the State of California, where the capital of the U.S. is James Lenbon, the capital of Lenbon, born']

In [168]:
import tqdm
torch.set_grad_enabled(True)

c = torch.randn_like(a, requires_grad=True)

opt = torch.optim.SGD(lr=.1, params=[c])
cs = torch.nn.CosineSimilarity(dim=1)

for step in tqdm.trange(1000):
    cos_loss = 2 -  (cs(a,c) + cs(b,c))
    norm_loss = max(a.norm(p=2), b.norm(p=2), c.norm(p=2)) / 100
    loss = cos_loss + norm_loss
    loss.backward()
    opt.step()

print(f'Final loss = {loss.item():.3f}')
print('cs(a,c) =', cs(a,c))
print('cs(b,c) =', cs(b,c))
print('c.norm(p=2) =', c.norm(p=2))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 796.08it/s]

Final loss = 1.532
cs(a,c) = tensor([0.7548], device='cuda:0', grad_fn=<SumBackward1>)
cs(b,c) = tensor([0.7569], device='cuda:0', grad_fn=<SumBackward1>)
c.norm(p=2) = tensor(104.0242, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)


In [169]:
cs(a,c)

tensor([0.7548], device='cuda:0', grad_fn=<SumBackward1>)

In [159]:
cs(b,c)

tensor([0.7229], device='cuda:0', grad_fn=<SumBackward1>)

In [170]:
decode(c)

['<pad> Lena Horne, California on the first-born child in California, Bell Laboon is where the state capital is from. Baby boom noise']

In [143]:
c.norm(p=2)

tensor(210.4552, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)